In [1]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number
import numpy as np
import string

import nltk
#
# you may need to run the following
nltk.download('stopwords')
nltk.download('wordnet')

# Enable inline plotting
%matplotlib inline 

# Location = r'/Users/soumyaparvatikar/JupyterProjects/hw1/censusData.csv'
Location = r'/Users/soumyaparvatikar/Desktop/project_dataset/train.csv'
df = pd.read_csv(Location)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/soumyaparvatikar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/soumyaparvatikar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df[:14]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [3]:
# add a row for clean comment
df['clean'] = None
vec = []
for i, row in df.iterrows():
    if((row['toxic'] == 1) or (row['severe_toxic'] == 1) or (row['obscene'] == 1) or (row['threat'] == 1) or (row['insult'] == 1) or (row['identity_hate'] == 1)):
        vec.append(0)
    else:
        vec.append(1)
df['clean'] = vec

In [4]:
df[:10]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,1
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,1
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,1
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,1
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,1


In [5]:
df.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805,0.898321
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420,0.302226
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
# stemming and tokenizing
from nltk.stem import WordNetLemmatizer
#
#  Input : dataframe with a column names 'text' which contains raw tweets (one per row)
#  Output: A list of lists of tokens corrsponding to the 'text' column
#
def tokenize_tweets(df):
    """Given a df with tweets in 'text' col, this function return tokens as a list of lists"""

    # apply tokenize to the 'text' coolumn in the tweets df
    tweet_tokenizer = nltk.tokenize.TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
    tokens = df['comment_text'].apply(tweet_tokenizer.tokenize)
    
    # filter
    misc = ['’', '…', '—', '"', 'w', '...', '️', 'http', 'https', '(', ')', '.','|', ',']
#     to_remove = nltk.corpus.stopwords.words('English') + list(string.punctuation) + misc
#     to_remove = misc
    to_remove = []
    
    lemmatizer = WordNetLemmatizer()
    # [[a,bird,flew], [high, main, france, the]] => [[bird,fly],[high,france,main]]
    tokens = [[lemmatizer.lemmatize(token) for token in tw if token not in to_remove] for tw in tokens]      
    return(tokens)

In [7]:
all_tokens = tokenize_tweets(df)
all_tokens[5]

['"',
 'congratulation',
 'from',
 'me',
 'a',
 'well',
 ',',
 'use',
 'the',
 'tool',
 'well',
 '.',
 '·',
 'talk',
 '"']

In [8]:
def removeEmptyComments(all_tokens, dframe):
    unwanted_indices = []
    desired_df = pd.DataFrame()
    count = 0
    filtered_tokens = []
    
    for idx, i in enumerate(all_tokens):  
        if len(i) == 0:
            count = count + 1
            unwanted_indices.append(idx)
#             df.drop(df.index[idx], inplace=True)
        else:
            filtered_tokens.append(i)
#             df = df.append(i)
    
    desired_indices = [j for j in dframe.index if j not in unwanted_indices]
    desired_df = dframe.iloc[desired_indices]
    desired_df = desired_df.reset_index(drop = True)
    print(count) 
    return filtered_tokens, desired_df

In [9]:
# remove empty comments
filtered_tokens, desired_df = removeEmptyComments(all_tokens, df)
print(filtered_tokens[:3])

0
[['explanation', 'why', 'the', 'edits', 'made', 'under', 'my', 'username', 'hardcore', 'metallica', 'fan', 'were', 'reverted', '?', 'they', "weren't", 'vandalism', ',', 'just', 'closure', 'on', 'some', 'gas', 'after', 'i', 'voted', 'at', 'new', 'york', 'doll', 'fac', '.', 'and', 'please', "don't", 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', "i'm", 'retired', 'now', '.', '89.205', '.', '38.27'], ["d'aww", '!', 'he', 'match', 'this', 'background', 'colour', "i'm", 'seemingly', 'stuck', 'with', '.', 'thanks', '.', '(', 'talk', ')', '21:51', ',', 'january', '11', ',', '2016', '(', 'utc', ')'], ['hey', 'man', ',', "i'm", 'really', 'not', 'trying', 'to', 'edit', 'war', '.', "it's", 'just', 'that', 'this', 'guy', 'is', 'constantly', 'removing', 'relevant', 'information', 'and', 'talking', 'to', 'me', 'through', 'edits', 'instead', 'of', 'my', 'talk', 'page', '.', 'he', 'seems', 'to', 'care', 'more', 'about', 'the', 'formatting', 'than', 'the', 'actual', 'info', '.']

In [10]:
# import Set

def separateValidationSet(filtered_tokens, dframe):
    trainDF = pd.DataFrame()
    validationDF = pd.DataFrame()
    validation_indices = []
    train_indices = []
    validationSet = []
    trainSet = []
    for idx, i in enumerate(filtered_tokens):
        if (idx%10 <= 2):
            validationSet.append(i)
            validation_indices.append(idx)
        else:
            trainSet.append(i)
            train_indices.append(idx)
    
#     trainDF_indices = [j for j in dframe.index if j in validation_indices]
#     trainDF = dframe.iloc[trainDF_indices]
    trainDF = dframe.iloc[train_indices]
    trainDF = trainDF.reset_index(drop = True) 
    
    validationDF = dframe.iloc[validation_indices]
    validationDF = validationDF.reset_index(drop = True)
    
    return trainSet, validationSet, trainDF, validationDF
        

In [11]:
trainSet_tokens, validationSet_tokens, trainDF, validationDF = separateValidationSet(filtered_tokens, desired_df)

In [12]:
# calculate the frequency of words
from sklearn.feature_extraction.text import CountVectorizer
newArray = []
newArray = [' '.join(i) for i in trainSet_tokens ]

count_vect = CountVectorizer(max_df=0.8, min_df=5, max_features=15000, ngram_range=(1, 2))  
counts_train = count_vect.fit_transform(newArray)
print(len(count_vect.get_feature_names()))


15000


In [13]:
# calculate the TFID score
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer().fit(counts_train)

counts_train = transformer.transform(counts_train) 


In [14]:
newArray_validation = []
# newArray = top_words.apply(lambda x: ' '.join(x))
newArray_validation = [' '.join(i) for i in validationSet_tokens ]

print(len(count_vect.get_feature_names()))

counts_validation = count_vect.transform(newArray_validation)

counts_validation = transformer.transform(counts_validation)

15000


In [15]:
# prediction for validation set
from sklearn.naive_bayes import MultinomialNB

model = {}

for i in range(1,8):
    y_train=trainDF.iloc[:,i+1].values 
    model[i] = MultinomialNB(alpha=1, fit_prior=True, class_prior=None).fit(counts_train, y_train)


In [16]:
# prediction for train set

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_fscore_support

for i in range(1,8):
    y_train=trainDF.iloc[:,i+1].values
    y_validation = validationDF.iloc[:,i+1].values
#     predicted = model[i].predict(counts_train)
    predicted = (model[i].predict_proba(counts_train)[:,1] >= 0.3).astype(bool)
    
    print("accuracy: ", np.mean(predicted == y_train))
    print(precision_recall_fscore_support(y_train, predicted, average='binary'))
    print("----------------------------------------------")

    

accuracy:  0.9515483576397282
(0.7623997623997624, 0.7188462615513862, 0.7399827039492648, None)
----------------------------------------------
accuracy:  0.9906355473191345
(0.5439642324888226, 0.33031674208144796, 0.411036036036036, None)
----------------------------------------------
accuracy:  0.9739209840732683
(0.7854158754272693, 0.6987666835614124, 0.73956191327671, None)
----------------------------------------------
accuracy:  0.9970098210368938
(0.5882352941176471, 0.02967359050445104, 0.05649717514124293, None)
----------------------------------------------
accuracy:  0.9704205051074763
(0.7253521126760564, 0.6502525252525253, 0.685752330226365, None)
----------------------------------------------
accuracy:  0.9912085157432027
(0.5260663507109005, 0.11178247734138973, 0.18438538205980068, None)
----------------------------------------------
accuracy:  0.9420496154844716
(0.9407803283615734, 0.9983255424544757, 0.968699074947171, None)
---------------------------------------

In [17]:
# prediction for validation set
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_fscore_support

for i in range(1,8):
    y_validation = validationDF.iloc[:,i+1].values    
#     predicted = model[i].predict(counts_validation)
    predicted = (model[i].predict_proba(counts_validation)[:,1] >= 0.3).astype(bool)
    
    print("accuracy: ", np.mean(predicted == y_validation))
    print(precision_recall_fscore_support(y_validation, predicted, average='binary'))
    print("----------------------------------------------")


accuracy:  0.9478191844919787
(0.7503004085556356, 0.6815105872080331, 0.714253031342942, None)
----------------------------------------------
accuracy:  0.9902657085561497
(0.5431654676258992, 0.3081632653061224, 0.3932291666666667, None)
----------------------------------------------
accuracy:  0.9724264705882353
(0.7790590405904059, 0.6675889328063241, 0.7190293742017881, None)
----------------------------------------------
accuracy:  0.9970755347593583
(0.6666666666666666, 0.014184397163120567, 0.027777777777777776, None)
----------------------------------------------
accuracy:  0.969167780748663
(0.7122337790985637, 0.6163737676810973, 0.6608455882352942, None)
----------------------------------------------
accuracy:  0.9912057152406417
(0.44, 0.08009708737864078, 0.135523613963039, None)
----------------------------------------------
accuracy:  0.9397142379679144
(0.9385970664743043, 0.9982099267697315, 0.9674860863883193, None)
----------------------------------------------


In [18]:
# merge train and validation and run again
newArray_complete = []
newArray_complete = [' '.join(i) for i in filtered_tokens ]

count_vect_complete = CountVectorizer(max_df=0.8, min_df=5, max_features=15000, ngram_range=(1, 2))  
counts_train_complete = count_vect_complete.fit_transform(newArray_complete)
print(len(count_vect_complete.get_feature_names()))
# -------------------------------------------------

transformer_complete = TfidfTransformer().fit(counts_train_complete)

counts_train_complete = transformer_complete.transform(counts_train_complete) 
# -------------------------------------------------

model_complete = {}

for i in range(1,8):
    y_train_complete=desired_df.iloc[:,i+1].values 
    model_complete[i] = MultinomialNB(alpha=1, fit_prior=True, class_prior=None).fit(counts_train_complete, y_train_complete)


15000


In [19]:
Loc_test = r'/Users/soumyaparvatikar/Desktop/project_dataset/test.csv'
df_test = pd.read_csv(Loc_test)

Loc_test_label = r'/Users/soumyaparvatikar/Desktop/project_dataset/test_labels.csv'
df_test_label = pd.read_csv(Loc_test_label)

In [20]:
df_test_label_clean = df_test_label[df_test_label['toxic'] != -1]
df_test_clean = df_test[df_test_label['toxic'] != -1]
df_test_label_clean.count()
print(df_test_label_clean.count())
print(df_test_clean.count())

id               63978
toxic            63978
severe_toxic     63978
obscene          63978
threat           63978
insult           63978
identity_hate    63978
dtype: int64
id              63978
comment_text    63978
dtype: int64


In [21]:
# add a row for clean comment
df_test_label_clean['clean'] = None
temp_vec = []
for i, row in df_test_label_clean.iterrows():
    if((row['toxic'] == 1) or (row['severe_toxic'] == 1) or (row['obscene'] == 1) or (row['threat'] == 1) or (row['insult'] == 1) or (row['identity_hate'] == 1)):
        temp_vec.append(0)
    else:
        temp_vec.append(1)
df_test_label_clean['clean'] = temp_vec

/Users/soumyaparvatikar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/soumyaparvatikar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [22]:
print(len(vec))
print(df_test_label_clean.count())

159571
id               63978
toxic            63978
severe_toxic     63978
obscene          63978
threat           63978
insult           63978
identity_hate    63978
clean            63978
dtype: int64


In [23]:
all_tokens_test = tokenize_tweets(df_test_clean)
all_tokens_test[228]

['"',
 ':',
 'maybe',
 'one',
 'of',
 'these',
 'day',
 '.',
 'cool',
 'random',
 'quote',
 ',',
 'by',
 'the',
 'way',
 ',',
 'at',
 '.',
 '"',
 '"',
 'like',
 '"',
 '"',
 'you',
 'fucking',
 'breeder',
 ',',
 'why',
 "don't",
 'you',
 'fuck',
 'your',
 'own',
 'sex',
 '?',
 '"',
 '"',
 '"']

In [24]:

print(len(all_tokens_test))
print(df_test_clean.count())

63978
id              63978
comment_text    63978
dtype: int64


In [25]:
newArray_test = []
newArray_test = [' '.join(i) for i in all_tokens_test ]

print(len(count_vect_complete.get_feature_names()))

counts_test = count_vect_complete.transform(newArray_test)

counts_test = transformer_complete.transform(counts_test)

15000


In [26]:
# test results
from sklearn.naive_bayes import MultinomialNB
print(desired_df.shape)
print(df_test_label_clean.shape)

print(sum(df_test_label_clean['toxic'] == 1))
print(sum(df_test_label_clean['toxic'] == 0))

# changes
for i in range(1,8):
#     y_train=desired_df.iloc[:,i+1].values
    y_test = df_test_label_clean.iloc[:,i].values

    # use the tfid score as x_train to train the model
    # changes
#     model = MultinomialNB(alpha=0, fit_prior=True, class_prior=None).fit(counts, y_train)
#     predicted = model.predict(counts_test)
    predicted = (model_complete[i].predict_proba(counts_test)[:,1] >= 0.3).astype(bool)
    print("accuracy: ", np.mean(predicted == y_test)) 
    print(precision_recall_fscore_support(y_test, predicted, average='binary'))
    print("----------------------------------------------")

(159571, 9)
(63978, 8)
6090
57888
accuracy:  0.9201913157647942
(0.5642969158389963, 0.7090311986863711, 0.6284383641391356, None)
----------------------------------------------
accuracy:  0.9910281659320391
(0.29504950495049503, 0.40599455040871935, 0.34174311926605505, None)
----------------------------------------------
accuracy:  0.9553752852543062
(0.6071245515120451, 0.6418314819832024, 0.6239957855919925, None)
----------------------------------------------
accuracy:  0.9966863609365719
(0.4444444444444444, 0.018957345971563982, 0.03636363636363637, None)
----------------------------------------------
accuracy:  0.9539060301978806
(0.5672481710748453, 0.588269623577473, 0.5775676837129351, None)
----------------------------------------------
accuracy:  0.9893713464003251
(0.5666666666666667, 0.19101123595505617, 0.2857142857142857, None)
----------------------------------------------
accuracy:  0.9353684078902123
(0.9434589800443459, 0.9875638694033082, 0.965007743147526, None)


Logistic regression

In [27]:
# logistic regression for training set
# Training set was evaluated only to compare the results with validation set and get a sense for overfitting.
from sklearn.linear_model import LogisticRegression

logreg = {}

for i in range(1,8):    
    y_train=trainDF.iloc[:,i+1].values 
    logreg[i] = LogisticRegression()
    logreg[i].fit(counts_train, y_train)
    lr_prediction = logreg[i].predict(counts_train)
    print("accuracy: ", np.mean(lr_prediction == y_train))
    print(precision_recall_fscore_support(y_train, lr_prediction, average='binary'))
    print("----------------------------------------------")

accuracy:  0.9613604419018971
(0.9396563573883161, 0.6381032390553533, 0.7600622637313763, None)
----------------------------------------------
accuracy:  0.9913338525859676
(0.6532438478747203, 0.2642533936651584, 0.3762886597938144, None)
----------------------------------------------
accuracy:  0.9789255051522395
(0.9342265529841657, 0.6479134989018416, 0.7651636073423782, None)
----------------------------------------------
accuracy:  0.9972694473540498
(0.7962962962962963, 0.12759643916913946, 0.21994884910485935, None)
----------------------------------------------
accuracy:  0.9742611840750589
(0.8716235032024505, 0.5645743145743146, 0.6852764094143404, None)
----------------------------------------------
accuracy:  0.9924439789075999
(0.7811320754716982, 0.2084592145015106, 0.329093799682035, None)
----------------------------------------------
accuracy:  0.9604293682127861
(0.9613723169875218, 0.9959633612741825, 0.9783621837549934, None)
--------------------------------------

In [28]:
# logistic regression for validation

for i in range(1,8):
    y_validation = validationDF.iloc[:,i+1].values 
#     logreg = LogisticRegression()
#     logreg.fit(counts_validation, y_validation)
    prediction = logreg[i].predict(counts_validation)
    print("accuracy: ", np.mean(prediction == y_validation))
    print(precision_recall_fscore_support(y_validation, prediction, average='binary'))
    print("----------------------------------------------")

accuracy:  0.9567179144385026
(0.9228176609369734, 0.5976860947391399, 0.7254901960784313, None)
----------------------------------------------
accuracy:  0.9903283756684492
(0.5721925133689839, 0.21836734693877552, 0.3161004431314623, None)
----------------------------------------------
accuracy:  0.9770429478609626
(0.9181765049678551, 0.6209486166007905, 0.7408630040084885, None)
----------------------------------------------
accuracy:  0.997305314171123
(0.7307692307692307, 0.1347517730496454, 0.2275449101796407, None)
----------------------------------------------
accuracy:  0.9718833556149733
(0.831876260928043, 0.5302186026575225, 0.6476439790575916, None)
----------------------------------------------
accuracy:  0.9925635026737968
(0.7745098039215687, 0.19174757281553398, 0.30739299610894943, None)
----------------------------------------------
accuracy:  0.9556525735294118
(0.9572208059394427, 0.9951179820992677, 0.9758015797932363, None)
--------------------------------------

In [34]:
# merge train and validation and run again
newArray_complete = []
newArray_complete = [' '.join(i) for i in filtered_tokens ]

count_vect_complete = CountVectorizer(max_df=0.8, min_df=5, max_features=150000, ngram_range=(1, 2))  
counts_train_complete = count_vect_complete.fit_transform(newArray_complete)
print(len(count_vect_complete.get_feature_names()))
# -------------------------------------------------

transformer_complete = TfidfTransformer().fit(counts_train_complete)

counts_train_complete = transformer_complete.transform(counts_train_complete) 
# -------------------------------------------------

logreg_complete = {}

for i in range(1,8):    
    y_train_complete=desired_df.iloc[:,i+1].values 
    logreg_complete[i] = LogisticRegression(C=1.85, penalty='l1')
    logreg_complete[i].fit(counts_train_complete, y_train_complete)
    

150000


In [35]:
newArray_test = []
# newArray = top_words.apply(lambda x: ' '.join(x))
newArray_test = [' '.join(i) for i in all_tokens_test ]

print(len(count_vect_complete.get_feature_names()))

counts_test = count_vect_complete.transform(newArray_test)

counts_test = transformer_complete.transform(counts_test)

150000


In [36]:
# Test results for Logistic regression

# from sklearn.naive_bayes import MultinomialNB
print(desired_df.shape)
print(df_test_label_clean.shape)

print(sum(df_test_label_clean['toxic'] == 1))
print(sum(df_test_label_clean['toxic'] == 0))

for i in range(1,8):
#     y_train=desired_df.iloc[:,i+1].values
    y_test = df_test_label_clean.iloc[:,i].values

    prediction = logreg_complete[i].predict(counts_test)

    print("accuracy: ", np.mean(prediction == y_test)) 
    print(precision_recall_fscore_support(y_test, prediction, average='binary'))
    print("----------------------------------------------")

(159571, 9)
(63978, 8)
6090
57888
accuracy:  0.9292725624433399
(0.5956952427540663, 0.7998357963875206, 0.682834513212308, None)
----------------------------------------------
accuracy:  0.9924974209884648
(0.36253041362530414, 0.40599455040871935, 0.3830334190231362, None)
----------------------------------------------
accuracy:  0.9648472912563694
(0.6924719701014416, 0.7027905716607965, 0.6975931155035633, None)
----------------------------------------------
accuracy:  0.9965613179530464
(0.47398843930635837, 0.3886255924170616, 0.42708333333333337, None)
----------------------------------------------
accuracy:  0.9647378786457845
(0.7037939436129481, 0.5900204260285964, 0.6419047619047619, None)
----------------------------------------------
accuracy:  0.990418581387352
(0.6073752711496746, 0.39325842696629215, 0.47740835464620635, None)
----------------------------------------------
accuracy:  0.9240207571352652
(0.9808475809385231, 0.9340434744955399, 0.9568735305859912, None)
-